In [41]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
import cirq
import tensorflow_quantum as tfq
import tensorflow as tf
from utilities.unitary_killer import UnitaryMurder
from utilities.variational import VQE
from utilities.simplifier import Simplifier
import sympy 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
#### let's define the unitary ####

unitary = np.array([[complex(.999887505, -1.49991876e-02), complex(-4.99732477e-05,-1.04730943e-06)], 
                    [complex(4.99732477e-05,-1.04730943e-06), complex(9.99887505e-01,1.49991876e-02)]])
print(cirq.is_unitary(unitary))
print(np.dot(unitary,np.transpose(np.conjugate(unitary))))

True
[[1.00000000e+00+0.j 6.03805177e-22+0.j]
 [6.03805177e-22+0.j 1.00000000e+00+0.j]]


In [39]:
bd= 1.5
problem_config ={"problem" : "H2", "geometry": [('H', (0., 0., 0.)), ('H', (0., 0., bd)), ('H', (0., 0., 2*bd)), ('H', (0., 0., 3*bd))], 
                  "multiplicity":1, "charge":0, "basis":"sto-3g"}
vqe_handler = VQE(n_qubits=8,problem_config=problem_config, optimizer="sgd", lr=.01,verbose=0)
uk = UnitaryMurder(vqe_handler, noise_config={})

In [45]:
simp = Simplifier(n_qubits=2)
simp.give_rz_rx_rz(unitary)

hey
hey
hey
hey
hey
hey


KeyboardInterrupt: 

In [63]:
gts=[cirq.rx, cirq.rz,cirq.rx]
symbols_names = [str(k) for k in range(3)]
symbols = [sympy.Symbol("{}".format(k)) for k in range(3)]
qubit = cirq.GridQubit.rect(1, 1)[0]
circuit = cirq.Circuit([g(sym).on(qubit) for g,sym in zip(gts, symbols)])
resolver = {s:k for s,k in zip(symbols_names, np.random.randn(len(symbols)))}

In [86]:
qc = tfq.convert_to_tensor([cirq.resolve_parameters(circuit,resolver)])
qc = tfq.convert_to_tensor([circuit])

In [88]:
tfq.layers.Unitary()(qc, symbol_names = list(resolver.keys()), symbol_values=[list(resolver.values())])

<tf.RaggedTensor [[[(0.9489362835884094-0.08517752587795258j), (0.11859248578548431+0.27964353561401367j)], [(-0.11859248578548431+0.27964353561401367j), (0.9489362835884094+0.08517751842737198j)]]]>

In [67]:
list(resolver.keys())
list(resolver.values())

[-1.234514317076239, 0.29307080588143236, 0.6613553874401257]

In [69]:
uo = tfq.get_unitary_op()(qc, list(resolver.keys()), [list(resolver.values())])

In [79]:
def loss(pred, target):
    #pred ---> tf ragged tensor
    return 1-0.25*np.trace(np.real(np.dot(np.conjugate(np.transpose(np.squeeze(pred.numpy()))), target))**2)

0.5486541329640926

In [82]:
class Ucompiler(tf.keras.Model):
    def __init__(self, symbols, observable, batch_sizes=1):
        """
        symbols: symbolic variable [sympy.Symbol]*len(rotations_in_circuit)
        batch_size: how many circuits you feed the model at, at each call (this might )
        """
        super(Ucompiler,self).__init__()
        self.unitary_layer = tfq.layers.Unitary()
        self.symbols = symbols
    def call(self, inputs):
        return self.unitary_layer()(inputs, symbol_names = self.symbols, symbol_values = self.trainable_variables)

In [ ]:
class Ucompiler(tf.keras.Model):
    def __init__(self, symbols, observable, batch_sizes=1):
        """
        symbols: symbolic variable [sympy.Symbol]*len(rotations_in_circuit)
        batch_size: how many circuits you feed the model at, at each call (this might )
        """
        super(Ucompiler,self).__init__()
        self.gradient_norm = Metrica(name="grad_norm")
        
    def call(self, inputs):
        """
        inputs: tfq circuits (resolved or not, to train one has to feed unresolved)
        """
        return self
        f = self.expectation_layer(feat, operators=self.observable, symbol_names=self.symbols)
        f = tf.math.reduce_sum(f,axis=-1)
        return f

    def train_step(self, data):
        x,y=data
        with tf.GradientTape() as tape:
            tape.watch(self.trainable_variables)
            preds = self(x,training=True)
            energy = self.compiled_loss(preds, preds) #notice that compiled loss takes care only about the preds
        train_vars = self.trainable_variables
        grads=tape.gradient(energy,train_vars)

        self.cost_value.update_state(energy)
        return {k.name:k.result() for k in self.metrics}
